## 🔍 Circuit and Index Constituent Filter
*   Author: Naved Iqbal
*   Email: navediqbal5@gmail.com
*   GitHub: https://github.com/Naved-Iqbal/StockTrading.git


# Overview & Instructions

---

This tool allows users to filter stocks based on two key criteria:

✅ 1. **Filter Low Circuit Stocks**

- **Purpose:** Exclude stocks with strict daily price bands (e.g. 5% circuit limit).
- **Included Stocks:** Only those with either `"No Band"` or circuit band greater than 5%.
- **Required File:** *Circuit Limit CSV* containing `Symbol` and `Band` columns.

**📥 How to download the Circuit Limit file:**

1. Visit the NSE’s official [**Daily Price Bands Report**](https://www.nseindia.com/regulations/daily-price-bands-reports) page.
2. Download the `.CSV` file by clicking on `Complete List of Price Bands (.csv)`
3. Use this file as input when the “Filter Low Circuit Stocks” option is selected.

---

✅ 2. **Filter Index Constituents**

- **Purpose:** Filter to include only stocks that are part of a selected index (e.g. NIFTY 500).
- **Required File:** *Index Constituents CSV* with at least a `Symbol` column.

**📥 How to download the Index Constituents file:**

1. Go to the [**NIFTY Indices page**](https://www.niftyindices.com/indices/).
2. Select an Index (e.g. Nifty 500)
2. Scroll down to the **“Downloads”** section
3. Click on `Index Constituent` to download the selected index constituent `.CSV` file.
4. Use this file when the “Filter Index Constituents” checkbox is selected.

---

⚠️ **Note:**

- You **must select at least one** filter (Low Circuit or Index Constituents) for the filtering process to execute.
- Uploading the appropriate CSV files is required based on selected options.
- Once filtered, results can be downloaded via:
  - **Download as CSV**
  - **Download as Excel**



In [75]:
import pandas as pd
from google.colab import files
import pytz
import io
import sys
import time
import ipywidgets as widgets

In [76]:
from textwrap import indent
# Adjust layout to increase width
# layout = widgets.Layout(width='50%')
style = {'description_width': 'initial'}

label_note = widgets.Label(value="IMPORTANT: The uploaded CSV file must have a column 'Symbol'\n\n")

# Create a file upload widget
label_stocks_file = widgets.Label(value="Upload Stocks CSV file:")
stocks_file = widgets.FileUpload(
    disabled=False,
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

filter_dropdwon = widgets.Dropdown(
    options=[('Low Circuit Stocks (2%, 5%)', 1), ('Index Constituent', 2), ('Both', 3)],
    value=2,
    description='Filter:',
    indent=False,
    style=style,
    layout=widgets.Layout(width='50%') # Added layout to control dropdown width
)

checkbox_low_circuit_stocks = widgets.Checkbox(
    value=False,
    description='Filter Low Circuit Stocks (2%, 5%)',
    disabled=False,
    indent=False
)

label_upload_circuit_stocks = widgets.Label(value="Upload Circuit Limit CSV File")
stocks_circuit_file = widgets.FileUpload(
    disabled=False,
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
)


checkbox_index_constituent = widgets.Checkbox(
    value=False,
    description='Filter Index Constituents',
    disabled=False,
    indent=False
)

label_index_constituents_stocks_file = widgets.Label(value="Upload Index Constituents CSV file:")
index_constituents_stocks_file = widgets.FileUpload(
    disabled=False,
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)


# Display the widgets
display(label_note,
        widgets.HBox([label_stocks_file, stocks_file]),
        widgets.HBox([checkbox_low_circuit_stocks, label_upload_circuit_stocks, stocks_circuit_file]),
        widgets.HBox([checkbox_index_constituent, label_index_constituents_stocks_file, index_constituents_stocks_file]))

Label(value="IMPORTANT: The uploaded CSV file must have a column 'Symbol'\n\n")

In [77]:
# prompt: check if at least one of the checkbox is selected, add condition if selected
button_filter = widgets.Button(description='Filter Stocks', style=style)
# Create buttons for downloading
download_csv_button = widgets.Button(description="Download as CSV")
download_excel_button = widgets.Button(description="Download as Excel")
output_widget = widgets.Output()

def on_button_clicked(b):
    with output_widget:

        output_widget.clear_output()
        df = None
        if stocks_file.value:
            uploaded_stocks_file = stocks_file.value[list(stocks_file.value.keys())[0]]['content']
            df = pd.read_csv(io.BytesIO(uploaded_stocks_file))
            print("\nStocks file loaded successfully.")
            if 'Symbol' not in df.columns:
                print("\nError: 'Symbol' column not found in Stocks file.")
                return
        else:
            print("\nPlease upload the Stocks CSV file.")
            return


        if not (checkbox_low_circuit_stocks.value or checkbox_index_constituent.value):
            print("\nPlease select at least one filter checkbox.")
            return


        if checkbox_low_circuit_stocks.value:
            df_circuit = None
            if stocks_circuit_file.value:
                uploaded_circuit = stocks_circuit_file.value[list(stocks_circuit_file.value.keys())[0]]['content']
                df_circuit = pd.read_csv(io.BytesIO(uploaded_circuit))
                print("\nCircuit Limit file loaded successfully.")
                if 'Symbol' not in df_circuit.columns:
                    print("\nError: 'Symbol' column not found in Circuit Limit file.")
                    return
            else:
                print("\nPlease upload the Circuit Limit CSV file if 'Filter Low Circuit Stocks' is selected.")
                return


            df = pd.merge(df, df_circuit[['Symbol', 'Band']], on='Symbol', how='left')

            # Create a new column with numeric conversion of 'Band'; non-numeric values become NaN
            df['Band_Numeric'] = pd.to_numeric(df['Band'], errors='coerce')

            # Define the conditions for removal
            df = df[(df['Band'] == 'No Band') | (df['Band_Numeric'] > 5)]

            df = df.drop(columns=['Band_Numeric'])


            print(f"\nFiltered for Low Circuit Stocks = {len(df)}")


        if checkbox_index_constituent.value:
            df_index = None
            if index_constituents_stocks_file.value:
                uploaded_index = index_constituents_stocks_file.value[list(index_constituents_stocks_file.value.keys())[0]]['content']
                df_index = pd.read_csv(io.BytesIO(uploaded_index))
                print("\nIndex Constituents file loaded successfully.")
                if 'Symbol' not in df_index.columns:
                    print("\nError: 'Symbol' column not found in Index Constituents file.")
                    return
            else:
                 print("\nPlease upload the Index Constituents CSV file if 'Filter Index Constituents' is selected.")
                 return

            # Apply index constituent filter
            common_symbols_index = pd.merge(df, df_index, on='Symbol', how='inner')
            df = common_symbols_index
            print(f"\nFiltered for Index Constituents = {len(df)}")

        # Function to download DataFrame as CSV
        def download_df_as_csv(df, filename="dataframe.csv"):
          df.to_csv(filename, index=False)
          files.download(filename)

        # Function to download DataFrame as Excel
        def download_df_as_excel(df, filename="dataframe.xlsx"):
          df.to_excel(filename, index=False)
          files.download(filename)


        # Display buttons
        # display(download_csv_button, download_excel_button)

        # Define click event handlers
        def on_csv_button_clicked(b):
            if df.shape[0] > 0:
                download_df_as_csv(df, filename="filtered_stocks.csv")
            else:
                print("\n0 filtered stocks found")

        def on_excel_button_clicked(b):
            if df.shape[0] > 0:
                download_df_as_excel(df, filename="filtered_stocks.xlsx")
            else:
                print("\n0 filtered stocks found")

        # Attach event handlers to buttons
        download_csv_button.on_click(on_csv_button_clicked)
        download_excel_button.on_click(on_excel_button_clicked)


button_filter.on_click(on_button_clicked)

# Display the button and output widget
display(button_filter, download_csv_button, download_excel_button, output_widget)


Button(description='Filter Stocks', style=ButtonStyle())

Button(description='Download as CSV', style=ButtonStyle())

Button(description='Download as Excel', style=ButtonStyle())

Output()